In [ ]:
import json
from bs4 import BeautifulSoup
import urllib3
import re
import pandas as pd
import numpy as np

In [ ]:
with open('all_usernames.csv') as file:
    usernames = file.read().splitlines()

In [ ]:
def get_html_data(username):
        html_data = pd.DataFrame(columns=['departamento','nombre','correo','tipo','horas','perfilAcademico'])
        url = "https://www.uchile.cl/portafolio-academico/perfilAcademico.jsf?username=" + username
        http = urllib3.PoolManager()
        response = http.request('GET', url)
        soup = BeautifulSoup(response.data.decode('utf-8'), "html.parser")
        try:
                perfilAcademico = soup.findAll('h7')[0].text
                other_temp_df = pd.DataFrame({'nombre':soup.findAll('h1')[0].text,
                                'correo':soup.findAll('a', attrs={'href': re.compile("mailto*")})[0].text,
                                'perfilAcademico':perfilAcademico},
                                index=[0])
                html_data = html_data.append(other_temp_df)
                print(username)
        except:
                try:
                        other_temp_df = pd.DataFrame({'nombre':soup.findAll('h1')[0].text,
                                        'correo':soup.findAll('a', attrs={'href': re.compile("mailto*")})[0].text},
                                        index=[0])
                        html_data = html_data.append(other_temp_df)
                except:
                        print(username)
                        return pd.DataFrame()
        nombramientos = soup.findAll('ul', attrs={'class': re.compile("nombramientos")})
        for i in nombramientos:
                for li in i.findAll('li'):
                        vals = {'tipo':li.findAll('h6')[0].text,
                                'departamento':li.findAll('h6')[1].text,
                                'horas':li.findAll('h6')[2].text}
                        temp_df = pd.DataFrame(vals,index=[0])
                        html_data = html_data.append(temp_df)
        return html_data


In [ ]:
def get_datatable(datatable, user):
    url = "https://www.uchile.cl/portafolio-academico/datatable-" + datatable + ".jsf?username=" + user
    http = urllib3.PoolManager()
    response = http.request('GET', url)
    soup = BeautifulSoup(response.data.decode('utf-8'))
    data = json.loads(soup.get_text())
    return data

In [ ]:
tables = ["titulosGrados","CargosCurriculum","docencia","publicaciones","libros","proyectosInv","tesis","patentes"]
titulosGrados = ['descripcion','institucion','fecha']
CargosCurriculum = ['fechaInicio','fechaFin','cargo','organismo','departamento']
docencia = ['organismo','areaAcademica','nombreRamo','codigo','agno','participacionDoc']
publicaciones = ['primerosAutores','tituloPublicacion','agno','tituloRevista','tipoPublicacion','doiURL']
libros = ['titulo','editor','anoPublicacion']
proyectosInv = ['codigoProyecto','tituloProyecto','fechaInicio','fechaFin','investigadorCoinvestigador','intituciones','programa','fuente']
tesis = ['autores','titulo','facultad','anoPublicacion','profesorGuia','profesorComision','tema','fuente']
patentes = ['prioridad','titulo','derecho','academicos','facultad_1','facultad_2','facultad_3','licencia','numSolicitud']
all_columns = []
all_columns.extend(titulosGrados)
all_columns.extend(CargosCurriculum)
all_columns.extend(docencia)
all_columns.extend(publicaciones)
all_columns.extend(libros)
all_columns.extend(proyectosInv)
all_columns.extend(tesis)
all_columns.extend(patentes)

In [ ]:
def read_json(json, columns, name):
    #if len(temp1['filasDataTable'] > 0):
    df = pd.DataFrame(columns=columns)
    for i in range(len(json['filasDataTable'])):
        for value in columns:
            df.loc[i, value] = json['filasDataTable'][i][value]
    return df

In [ ]:
data = []
dataframe = pd.DataFrame()
for i in range(1500,1720):
    try:
        user = usernames[i]
        temp1 = get_datatable(tables[0], user)
        temp2 = get_datatable(tables[1], user)
        temp3 = get_datatable(tables[2], user)
        temp4 = get_datatable(tables[3], user)
        temp5 = get_datatable(tables[4], user)
        temp6 = get_datatable(tables[5], user)
        temp7 = get_datatable(tables[6], user)
        temp8 = get_datatable(tables[7], user)
        try:
            temp_dataframe = pd.concat([get_html_data(user).reset_index(drop=True),
                                        read_json(temp1, titulosGrados, 'titulosGrados'),
                                        read_json(temp2, CargosCurriculum, 'CargosCurriculum'),
                                        read_json(temp3, docencia, 'docencia'),
                                        read_json(temp4, publicaciones, 'publicaciones'),
                                        read_json(temp5, libros, 'libros'),
                                        read_json(temp6, proyectosInv, 'proyectosInv'),
                                        read_json(temp7, tesis, 'tesis'),
                                        read_json(temp8, patentes, 'patentes')],
                                        axis=1)
            dataframe = dataframe.append(temp_dataframe)
        except:
            pass
    except:
        print(user)
        pass

In [ ]:
dataframe = dataframe.replace(np.nan, '', regex=True)
dataframe.to_csv('data.csv', index=False,encoding='utf-8-sig')